In [65]:
import numpy as np

sem_dir = "../sab_work/"
out_dir = "../data/sem_data/"
words = ["accident", "brilliant", "float", "excess", "promise"]
remap_words = ["accident-n", "brilliant-a", "float-v", "excess-n", "promise-v"]
remap_dict = {}
for i in range(len(words)):
    remap_dict[words[i]] = remap_words[i]
sets = ['train', 'test']

In [33]:
###get data in and map to a dictionary of dictionaries. Format is dictionary[feature][set] where feature is the word
###and set is like accident_train which returns the list of rows that have that feature in that set

word = words[0]
set_type = sets[0]

column_dict = {}
total_rows = {}
for word in words:
    for set_type in sets:
        with open(sem_dir + "sem_" +  word + "_" + set_type + ".txt") as f:
            num_rows = 0
            for i, line in enumerate(f.readlines()):
                for w1 in line.split(" "):
                    second_level = word + "_" + set_type
                    w = w1.strip()
                    column_dict[w] = column_dict.get(w, {})
                    column_dict[w][second_level] = column_dict[w].get(second_level, []) + [i]
                num_rows += 1
            total_rows[word + "_" + set_type] = num_rows

In [71]:
###explode the dictionary format to numpy arrays of same number of columns

n_features = len(column_dict.keys())
np_dict = {}
for s in total_rows.items():
    np_dict[s[0]] = np.zeros((s[1], n_features))

for i, feature in enumerate(column_dict.keys()):
    for data_set in column_dict[feature].keys():
        np_array = np_dict[data_set]
        for row in column_dict[feature][data_set]:
            np_array[row, i] = 1
        np_dict[data_set] = np_array

In [72]:
###write to disk

for data_set in np_dict.items():
    word_type = remap_dict[data_set[0].split("_")[0]]
    set_type = data_set[0].split("_")[1]
    np.save(out_dir + word_type + "_" + set_type + "_sem.npy", data_set[1])